# Top 70% marginal tax rate



## Setup

### Imports

In [19]:
import taxcalc as tc
import behresp as br

### Define reform

From https://apps.ospc.org/taxbrain/38482/.

In [1]:
THRESHOLD = 10000000.0  # 10 million.
RATE = 0.7

# List for each of the five MARS categories.
threshold_mars = [[THRESHOLD, THRESHOLD, THRESHOLD, THRESHOLD, THRESHOLD]]

reform = {
    2019: {
        '_CG_brk3': threshold_mars,
        '_CG_rt4': [RATE],
        '_AMT_CG_brk3': threshold_mars,
        '_AMT_CG_rt4': [RATE],
        '_II_brk7': threshold_mars,
        '_II_rt8': [RATE],
        '_PT_brk7': threshold_mars,
        '_PT_rt8': [RATE]
    }
}

In [2]:
from taxcalc import *

# use publicly-available CPS input file
recs = Records.cps_constructor()

# specify Calculator object for static analysis of current-law policy
pol = Policy()
calc1 = Calculator(policy=pol, records=recs)

cyr = 2020

# calculate current-law tax liabilities for cyr
calc1.advance_to_year(cyr)
calc1.calc_all()

# calculate marginal tax rate wrt cash charitable giving
(_, _, mtr1) = calc1.mtr('e19800', calc_all_already_called=True,
                         wrt_full_compensation=False)


# specify Calculator object for static analysis of reform policy
pol.implement_reform(reform)
calc2 = Calculator(policy=pol, records=recs)

# calculate reform tax liabilities for cyr
calc2.advance_to_year(cyr)
calc2.calc_all()

# calculate marginal tax rate wrt cash charitable giving
(_, _, mtr2) = calc2.mtr('e19800', calc_all_already_called=True,
                         wrt_full_compensation=False)

# extract variables needed for quantity_response utility function
# (note the aftertax price is 1+mtr because mtr wrt charity is non-positive)
vdf = calc1.dataframe(['s006', 'e19800', 'e00200'])
vdf['price1'] = 1.0 + mtr1
vdf['price2'] = 1.0 + mtr2
vdf['atinc1'] = calc1.array('aftertax_income')
vdf['atinc2'] = calc2.array('aftertax_income')

# group filing units into earnings groups with different response elasticities
# (note earnings groups are just an example based on no empirical results)
earnings_bins = [-9e99, 50e3, 9e99]  # two groups: below and above $50,000
vdf = add_income_table_row_variable(vdf, 'e00200', earnings_bins)
gbydf = vdf.groupby('table_row', as_index=False)

# compute percentage response in charitable giving
# (note elasticity values are just an example based on no empirical results)
price_elasticity = [-0.1, -0.4]
income_elasticity = [0.1, 0.1]
print('\nResponse in Charitable Giving by Earnings Group')
results = '{:18s}\t{:8.3f}\t{:8.3f}\t{:8.2f}'
colhead = '{:18s}\t{:>8s}\t{:>8s}\t{:>8s}'
print(colhead.format('Earnings Group', 'Num(#M)', 'Resp($B)', 'Resp(%)'))
tot_funits = 0.
tot_response = 0.
tot_baseline = 0.
idx = 0
for grp_interval, grp in gbydf:
    funits = grp['s006'].sum() * 1e-6
    tot_funits += funits
    response = quantity_response(grp['e19800'],
                                 price_elasticity[idx],
                                 grp['price1'],
                                 grp['price2'],
                                 income_elasticity[idx],
                                 grp['atinc1'],
                                 grp['atinc2'])
    grp_response = (response * grp['s006']).sum() * 1e-9
    tot_response += grp_response
    grp_baseline = (grp['e19800'] * grp['s006']).sum() * 1e-9
    tot_baseline += grp_baseline
    pct_response = 100. * grp_response / grp_baseline
    glabel = '[{:.8g}, {:.8g})'.format(grp_interval.left, grp_interval.right)
    print(results.format(glabel, funits, grp_response, pct_response))
    idx += 1
pct_response = 100. * tot_response / tot_baseline
print(results.format('ALL', tot_funits, tot_response, pct_response))

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.
You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.

Response in Charitable Giving by Earnings Group
Earnings Group    	 Num(#M)	Resp($B)	 Resp(%)
[-9e+99, 50000)   	 110.061	   0.000	    0.00
[50000, 9e+99)    	  57.451	   0.001	    0.00
ALL               	 167.512	   0.001	    0.00


### Initialize records and policy

In [21]:
# use publicly-available CPS input file
recs = tc.Records.cps_constructor()
pol = tc.Policy()

In [17]:
pol.implement_reform(reform=reform)

## Analysis

In [25]:
base = tc.Calculator(policy=pol, records=recs)

base.advance_to_year(2019)
base.calc_all()
itax_rev1 = base.weighted_total('iitax')

You loaded data for 2014.
Tax-Calculator startup automatically extrapolated your data to 2014.


In [ ]:

# specify baseline Calculator object representing current-law policy



cyr = 2020

# calculate aggregate current-law income tax liabilities for cyr
calc1.advance_to_year(cyr)
calc1.calc_all()
itax_rev1 = calc1.weighted_total('iitax')

# read JSON reform file
reform_filename = './ingredients/reformA.json'
params = Calculator.read_json_param_objects(reform=reform_filename,
                                            assump=None)

# specify Calculator object for static analysis of reform policy
pol.implement_reform(params['policy'])
calc2 = Calculator(policy=pol, records=recs)

# calculate reform income tax liabilities for cyr under static assumptions
calc2.advance_to_year(cyr)
calc2.calc_all()
itax_rev2sa = calc2.weighted_total('iitax')

# specify behavioral-response assumptions
behresp_json = '{"BE_sub": {"2018": 0.25}}'
behresp_dict = Calculator.read_json_assumptions(behresp_json)

# specify Calculator object for analysis of reform with behavioral response
calc2 = Calculator(policy=pol, records=recs)
calc2.advance_to_year(cyr)
_, df2br = behresp.response(calc1, calc2, behresp_dict)

# calculate reform income tax liabilities for cyr with behavioral response
itax_rev2br = (df2br['iitax'] * df2br['s006']).sum()

# print total income tax revenue estimates for cyr
# (estimates in billons of dollars rounded to nearest hundredth of a billion)
print('{}_CURRENT_LAW_P__itax_rev($B)= {:.3f}'.format(cyr, itax_rev1 * 1e-9))
print('{}_REFORM_STATIC__itax_rev($B)= {:.3f}'.format(cyr, itax_rev2sa * 1e-9))
print('{}_REFORM_DYNAMIC_itax_rev($B)= {:.3f}'.format(cyr, itax_rev2br * 1e-9))

# create multi-year diagnostic tables for
# (1) baseline,
# (2) reform excluding behavioral responses, and
# (3) reform including behavioral responses
num_years = 3  # number of diagnostic table years beginning with cyr
dtable1 = calc1.diagnostic_table(num_years)
dtable2 = calc2.diagnostic_table(num_years)
dvar_list3 = list()
year_list3 = list()
for year in range(cyr, cyr + num_years):
    calc1.advance_to_year(year)
    calc2.advance_to_year(year)
    _, df2br = behresp.response(calc1, calc2, behresp_dict)
    dvar_list3.append(df2br)
    year_list3.append(year)
dtable3 = create_diagnostic_table(dvar_list3, year_list3)
print()
print('DIAGNOSTIC TABLE FOR BASELINE')
print(dtable1)
print('DIAGNOSTIC TABLE FOR REFORM EXCLUDING BEHAVIORAL RESPONSES')
print(dtable2)
print('DIAGNOSTIC TABLE FOR REFORM INCLUDING BEHAVIORAL RESPONSES')
print(dtable3)